
**Student names:  Rohan Katkar**

**Student numbers: 101005880** 



# Discussion of Solution
Your notebook must begin with **this** text cell containing a description of your solution. In this discussion, include links to any resources that you used in developing your solution. Use proper MarkDown syntax to format your discussion.**This description should be approximately 500 words in length and cover the following:**

1. Which machine learning approach did you use?
The ML approach used is CNN, using convolution, pooling and RELU layers

2. How did you split your data between training and testing? 
_(e.g. hold-out test, cross-validation, repeated bootstrap samples, etc)_
We split our data into 2 test sets, training and testing. We split 20% of the entire dataset into the testing and 80% for training. 
 

3. How did you train your classifier?
We trained our model with 50 epochs with 50 steps per epoch with batch sizes of 50.

4. How did you estimate your future performance _(worst recall, best precision, overall accuracy)_?

   **Here are our final predictions:**
   1. Building with worst recall: RB
   2. Building with best precision: DT
   3. Overall accuracy: 0.07258065

5. Discuss the performance of your model. Which buildings did it do the best/worst on and why? What are the strengths and limitations of your method. 
The performance of our model was not very strong. But the greatest percision was for Dunton tower. This building has specific features, helping recognizing the building from all angles.
It did not predict well for Robertson and Southam. Given that their building features are quite similar in terms of height and structure. 

6. What would you have done differently if you had more time?
If we had more time on our hands we could have experimented with different models and tweak the hyperparameters to see the differences in our accuracy and precision. With exams and assignments due at the end of the term, it was a bit challenging to allocate time for this assignment.

#Required functions to test your method
_These are the five required methods that you must implement._

## prepareModel()
This function should prepare your model for multiple invocations of classifyImage(fname). For example, this function could be used to load a pre-trained model from a URL, where that model is then used by  classifyImage(fname). You should use global variables for any variables initialized by this function.

Runtime of this method is **limited to 5 minutes**, so please don’t retrain your network here. All training should be captured in a pre-trained model to be loaded by this method.


In [0]:
!wget https://github.com/rohank07/ML_A3/blob/master/model.zip?raw=true
!unzip model.zip?raw=true

from keras.models import load_model
from google.colab import files


In [0]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import load_model
# List of building codes to use throughout notebook.
buildingCodes = ('AA', 'CB', 'CT', 'DT', 'FH', 'HP', 'HS', 'LB', 'MC', 'ME', 'ML', 'PA', 'RB', 'RO', 'SA', 'TB')

def prepareModel():
  global model # Sample global variable that you may populate here.
  model = load_model('model/model.h5')


## label = classifyImage(fname)
Accepts a filename (e.g. ‘test/ME/testImage1.jpg’) of a square JPG image with size at least 500x500 pixels.
Returns a 2-character label corresponding to the predicted building (see table of labels above)

Any variables initialized by prepareModel() should be declared as global within this function if you want to access them (e.g. a pre-trained model)


In [0]:
def classifyImage(fname):
  global model
  tdf = pd.DataFrame(columns=["Image","label"])
  tdf.loc[0] = [fname,""]
  pred_gen = ImageDataGenerator(rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2)
  predict_generator = pred_gen.flow_from_dataframe(
      tdf,
      x_col= "x_col", 
      y_col = "y_col", 
      target_size=(500,500), 
      batch_size=50)
  print("Predicting class of '{0:s}' using model '{1:s}'".format(fname,model))
  predict = model.predict(predict_generator)
  return buildingCodes[np.argmax(predict[0])] 

## label = worstRecall()
Returns the label of a building that you expect will have to lowest recall, when tested on new images

In [0]:
def worstRecall():
  return buildingCodes[12] #Default behaviour to override


## label = bestPrecision()
Returns the label of a building that you expect will have to highest precision, when tested on new images

In [0]:
def bestPrecision():
  return buildingCodes[3] #Default behaviour to override




```
# This is formatted as code
```

##acc_score = estimatedAccuracy()
Returns the accuracy (between [0.0,1.0]) that you expect to achieve across all test images, assuming that each building is equally represented


In [0]:
# Function to return estimated accuracy that will be obtained across all test images
def estimatedAccuracy():
  return 0.07258065 

##acc_score = estimatedAccuracy()
Returns the accuracy (between [0.0,1.0]) that you expect to achieve across all test images, assuming that each building is equally represented


In [0]:
# Function to return estimated accuracy that will be obtained across all test images
def estimatedAccuracy():
  return 1.0 #Default behaviour to override

# Test required functions
_We will replace the text below with our actual test code..._

In [0]:
import numpy as np
from PIL import Image, ExifTags
import matplotlib.pyplot as plt
from glob import glob


# First prepare the model:
prepareModel() # Limited to 5 minutes...

# Load (secret) test data into local Colab environment
!wget https://github.com/jrgreen7/SYSC4906/blob/master/Assignments/Assignment3/Images/SampleImages.zip?raw=true
!unzip SampleImages.zip?raw=true #Should create 4 images
test_images = sorted(glob('SampleImages/*.jpg'))
actual_labels = (buildingCodes[0],buildingCodes[1],buildingCodes[3],buildingCodes[6])

# Classify sample test images:
TP = 0
for imgFname,actual_label in (zip(test_images,actual_labels)):
  pred_label = classifyImage(imgFname) # Predict the label of this image file

  # Plot the image with actual and predicted labels
  # Note that we may have to rotate the image, depending on the 
  # orientation of the camera. Use EXIF tags for this:
  im = Image.open(imgFname)
  for orientation in ExifTags.TAGS.keys() : 
    if ExifTags.TAGS[orientation]=='Orientation' : break 
  exif=dict(im._getexif().items())

  if exif[orientation] == 3 : 
    im=im.rotate(180, expand=True)
  elif exif[orientation] == 6 : 
    im=im.rotate(270, expand=True)
  elif exif[orientation] == 8 : 
    im=im.rotate(90, expand=True)

  im = im.convert('RGB') # May not be necessary?

  plt.title("Sample test image of {0:s} predicted as {1:s}".format(actual_label, pred_label))
  plt.imshow(np.asarray(im))
  plt.axis('off')
  plt.show()

  if (pred_label==actual_label):
    print('Correct!')
    TP += 1
  else:
    print("Incorrect...")

# Print the predicted performance:
print("Expected that worst recall would be on {0:s}".format(worstRecall()))
print("Expected that best precision would be on {0:s}".format(bestPrecision()))
print("Expected total accuracy would be {0:.3f}".format(estimatedAccuracy()))
print("Actual total accuracy is {0:.3f}".format((TP)/(len(test_images))))
